In [ ]:
import subprocess
from pydub import AudioSegment

def extract_audio_from_video(video_path, audio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command)

In [2]:
extract_audio_from_video("./files/itsub.mp4", "./files/itsub.mp3")